In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install hydra-core==1.1

In [ ]:
!pip install scipy==1.7.3

BRANCH = 'main'
# If you're using Google Colab and not running locally, uncomment and run this cell.
!apt-get install sox libsndfile1 ffmpeg
!pip install wget unidecode pynini==2.1.4
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

In [ ]:
from nemo.collections.tts.models import FastPitchModel

/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)
[NeMo W 2022-07-31 14:58:08 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-07-31 14:58:08 experimental:28] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [ ]:
!mkdir -p tts_dataset_files && cd tts_dataset_files \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tts_dataset_files/cmudict-0.7b_nv22.07 \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tts_dataset_files/heteronyms-030921 \
&& wget wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/nemo_text_processing/text_normalization/en/data/whitelist/lj_speech.tsv \
&& cd ..


Twilight Sparkle

In [ ]:
!cp /content/drive/MyDrive/TTS_finetuning/Twilight_sparkle/FastPitch--v_loss=1.2314-epoch=356-last.ckpt .
checkpoint = '/content/FastPitch--v_loss=1.2314-epoch=356-last.ckpt'
spec_model = FastPitchModel.load_from_checkpoint(checkpoint)
spec_model.save_to('twilight_sparkle_FastPitch_model.nemo')

Celestia

In [ ]:
!cp /content/drive/MyDrive/TTS_finetuning/Celestia/FastPitch--v_loss=1.2484-epoch=853-last.ckpt .
checkpoint = '/content/FastPitch--v_loss=1.2484-epoch=853-last.ckpt'
spec_model = FastPitchModel.load_from_checkpoint(checkpoint)
spec_model.save_to('celestia_FastPitch_model.nemo')

Applejack

In [ ]:
!cp /content/drive/MyDrive/TTS_finetuning/apple_jack/FastPitch--v_loss=1.1907-epoch=132-last.ckpt .
checkpoint = '/content/FastPitch--v_loss=1.1907-epoch=132-last.ckpt'
spec_model = FastPitchModel.load_from_checkpoint(checkpoint)
spec_model.save_to('applejack_FastPitch_model.nemo')

Spike

In [ ]:
# !cp /content/drive/MyDrive/TTS_finetuning/Twilight_sparkle/FastPitch--v_loss=1.2314-epoch=356-last.ckpt .
# checkpoint = '/content/FastPitch--v_loss=1.2314-epoch=356-last.ckpt'
# spec_model = FastPitchModel.load_from_checkpoint(checkpoint)
# spec_model.save_to('spike_FastPitch_model.nemo')

Trixie

In [12]:
!cp /content/tts_dataset_files/cmudict-0.7b_nv22.07 /content/tts_dataset_files/cmudict-0.7b_nv22.01

In [ ]:
!cp /content/drive/MyDrive/TTS_finetuning/trixie_02/FastPitch--v_loss=1.4934-epoch=496-last.ckpt .
checkpoint = '/content/FastPitch--v_loss=1.4934-epoch=496-last.ckpt'
spec_model = FastPitchModel.load_from_checkpoint(checkpoint)
spec_model.save_to('trixie_FastPitch_model.nemo')

HiFiGAN

In [ ]:
from nemo.collections.tts.models import HifiGanModel
from pathlib import Path

home_path = !(echo $HOME)
home_path = home_path[0]

HifiGanModel.from_pretrained("tts_hifigan")
nemo_files = [p for p in Path(f"{home_path}/.cache/torch/NeMo/").glob("**/tts_hifigan.nemo")]
print(f"Copying {nemo_files[0]} to ./")
Path("./tts_hifigan.nemo").write_bytes(nemo_files[0].read_bytes())

In [16]:
!rm *.ckpt

Uploading to the GCP bucket for easy access

In [22]:
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'convai-services-tech'
!gcloud config set project {project_id}

Updated property [core/project].


In [24]:
bucket_name = 'tts-voices-mlp'

# Full reference: https://cloud.google.com/storage/docs/gsutil/commands/mb
!gsutil mb gs://{bucket_name}

Creating gs://tts-voices-mlp/...


In [25]:
# Copy the file to our new bucket.
# Full reference: https://cloud.google.com/storage/docs/gsutil/commands/cp
!gsutil cp /content/applejack_FastPitch_model.nemo gs://{bucket_name}/
!gsutil cp /content/celestia_FastPitch_model.nemo gs://{bucket_name}/
!gsutil cp /content/trixie_FastPitch_model.nemo gs://{bucket_name}/
!gsutil cp /content/twilight_sparkle_FastPitch_model.nemo gs://{bucket_name}/
!gsutil cp /content/tts_hifigan.nemo gs://{bucket_name}/

Copying file:///content/applejack_FastPitch_model.nemo [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/
Operation completed over 1 objects/178.4 MiB.                                    
Copying file:///content/celestia_FastPitch_model.nemo [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large